##### Prerequisites 

# A case study on deforestation through illegal amber mining in Ukraine


##### Abstract

## 1. Introduction

## 2. Data and Methods

### 2.1 Data

### 2.2 Methods
#### 2.2.1 Preparation

In [1]:
# third party libs
import os
import re
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
from collections import namedtuple
from shapely.geometry import Point
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from bokeh.plotting import show, figure, gridplot, output_notebook, ColumnDataSource

# custom libs
from src.landsat import LandsatArchive
from src.utils import (get_data_dir,
                       bi,
                       ndvi,
                       RANDOM,
                       l7_radiance,
                       l7_reflectance,
                       write,
                       clip_raster,
                       reproject_from,
                       reproject_like,
                       draw_raster_sample,
                       confusion_matrix)


# make source data folders
directories = """
data
data.core
data.proc
data.prep
data.arch
"""

for item in directories.split():
    path = os.sep.join(item.split('.'))
    try:
        os.mkdir(path)
    except OSError:
        pass

# convenient access to data dir
DIRS = get_data_dir(str(Path('data').resolve()))

# pyproj definition of WGS84
WGS84 = {'init': 'epsg:4326'}

# area of interest
Bounds = namedtuple('Bounds', 'left bottom right top')
AIO = Bounds(25.6, 51.0, 27.6, 51.8)

# total number of threads to use
THREADS = 4

# force bokeh notebook output
output_notebook()

Loading BokehJS ...

#### 2.2.2 Tree cover reference layer

In [2]:
# load 
src = DIRS.core / 'l7_2000' # l7_200

if src.is_dir():
    l7_2000 = LandsatArchive.read(src)
else:
    l7_2000 = LandsatArchive.read(DIRS.arch / 'LE07_L1TP_184024_20000612_20170211_01_T1.tar.gz',
                                  extract_to=DIRS.core / 'l7_2000')
    
print(l7_2000)


        Spacecraft: LANDSAT_7
        Sensor: ETM
        Date acquired: 2000-06-12
        Cloud cover: 1.0
        Quality: 9
        


In [3]:
# compute band metrics and compose band stack
regex = re.compile(r'.*LE07.*_B(\d)\.TIF')

band_stack = []
for name in 'red green blue nir'.split():
    img = l7_2000[name]
    img_data = img.read(1)
    band_idx = regex.match(img.name).group(1)
        
    RMIN = l7_2000.metadata.get('MIN_MAX_RADIANCE', 'RADIANCE_MINIMUM_BAND_%s' % band_idx)
    RMAX = l7_2000.metadata.get('MIN_MAX_RADIANCE', 'RADIANCE_MAXIMUM_BAND_%s' % band_idx)
    QCMIN = l7_2000.metadata.get('MIN_MAX_PIXEL_VALUE', 'QUANTIZE_CAL_MIN_BAND_%s' % band_idx)
    QCMAX = l7_2000.metadata.get('MIN_MAX_PIXEL_VALUE', 'QUANTIZE_CAL_MAX_BAND_%s' % band_idx)
    ESD = l7_2000.metadata.get('IMAGE_ATTRIBUTES', 'EARTH_SUN_DISTANCE')
    SE = l7_2000.metadata.get('IMAGE_ATTRIBUTES', 'SUN_ELEVATION')

    radiance = l7_radiance(img_data, QCMIN, QCMAX, RMIN, RMAX, src_nodata=0)
    reflectance = l7_reflectance(radiance, ESD, SE, int(band_idx), src_nodata=0.0)
    
    band_stack.append(reflectance)
    img.close()

ndvi_data = ndvi(band_stack[0], band_stack[3])

band_stack.append(ndvi_data)
rgbn_data = np.array(band_stack, dtype=np.float32)

In [4]:
# reproject and clip
img = l7_2000['red']
crs = img.crs
transform = img.transform
img.close()

rgbn_path = write(rgbn_data, str(DIRS.proc / 'reference_image.tif'), 
                  driver='GTiff', transform=transform, 
                  crs=crs)

reproject = reproject_from(rgbn_path, WGS84, rgbn_path)

clip, transform = clip_raster(reproject, AIO)

clip_path = write(clip, reproject, 
                  driver='GTiff', transform=transform, 
                  crs=WGS84)

In [5]:
# sample image
ref_img = rasterio.open(clip_path, 'r')
ref_data = ref_img.read()
transform = ref_img.transform

sample = draw_raster_sample(ref_data, affine=transform, samples=500,
                            columns=['red', 'green', 'blue', 'nir', 'ndvi'])

points = [Point(coor.x, coor.y) 
          for idx, coor in sample[['x', 'y']].iterrows()]

geometry = gpd.GeoSeries(points)
geo_df = gpd.GeoDataFrame(sample, geometry=geometry)
geo_df.crs = WGS84

geo_df.to_file(str(DIRS.proc / 'samples.shp'))

In [6]:
# train rf
samples = gpd.read_file(str(DIRS.prep / 'forest.shp'))
samples['is_train'] = RANDOM.uniform(size=len(samples)) <= 0.75
train, test = samples[samples.is_train == True], samples[samples.is_train == False]

features = train.columns[:5]

clf = RandomForestClassifier(n_jobs=THREADS, random_state=RANDOM, n_estimators=1000, verbose=1)

clf.fit(train[features], train.label)

assessment = clf.predict(test[features])

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    0.6s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    0.2s finished


In [7]:
# classification
px_matrix = ref_data.T.reshape((ref_data.shape[1]*ref_data.shape[2],ref_data.shape[0]))
df = pd.DataFrame.from_records(px_matrix, columns=features)

classified = clf.predict(df[features])

classified = np.reshape(classified, (ref_data.shape[2], ref_data.shape[1])).T
classified = classified.astype(np.uint8)

treecover_2000 = write(classified, str(DIRS.proc / 'treecover_2000.tif'), crs=WGS84, driver='GTiff',
                       transform=transform, compress='lzw')

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    7.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   34.6s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:  3.0min finished


In [8]:
df = pd.DataFrame.from_records(confusion_matrix(test.label, assessment),
                               columns=['Condition positive', 'Condition negative'],
                               index=['Prediction positive', 'Prediction negative'])
df.to_csv(str(DIRS.proc / 'assess.csv'))

#### 2.2.3 Detection of amber mining sites

- classify and consider only classes over 90 % probability
- eliminate all where bi or ndvi is -1, 0, or 1

In [10]:
# load
if len(os.listdir(str(DIRS.core))) > 1:
    names = os.listdir(str(DIRS.core))
    is_archive = False

else:
    names = os.listdir(str(DIRS.arch))
    is_archive = True

landsat = []
for name in names:
    if is_archive:
        head = name.split('.')[0]
        obj = LandsatArchive.read(DIRS.arch / name, extract_to=DIRS.core / head)
    
    else:
        obj = LandsatArchive.read(DIRS.core / name)
    
    spacecraft = obj.metadata.get('product_metadata', 'spacecraft_id') 
    year = obj.metadata.get('product_metadata', 'date_acquired').split('-')[0]
    
    obj.alias = year
    
    if spacecraft == 'LANDSAT_7' and int(year) > 2000:
        landsat.append(obj)

landsat = sorted(landsat, key=lambda x: x.alias)

In [11]:
# compute band metrics ndvi and bi
regex = re.compile(r'.*LE07.*_B(\d)\.TIF')

images = []
for data in landsat:
    img = data['red']
    crs = img.crs
    transform = img.transform
    img.close()
    
    band_stack = []
    
    for name in 'red blue nir swir2'.split():
        img = data[name]
        img_data = img.read(1)
        band_idx = regex.match(img.name).group(1)
        
        RMIN = data.metadata.get('MIN_MAX_RADIANCE', 'RADIANCE_MINIMUM_BAND_%s' % band_idx)
        RMAX = data.metadata.get('MIN_MAX_RADIANCE', 'RADIANCE_MAXIMUM_BAND_%s' % band_idx)
        QCMIN = data.metadata.get('MIN_MAX_PIXEL_VALUE', 'QUANTIZE_CAL_MIN_BAND_%s' % band_idx)
        QCMAX = data.metadata.get('MIN_MAX_PIXEL_VALUE', 'QUANTIZE_CAL_MAX_BAND_%s' % band_idx)
        ESD = data.metadata.get('IMAGE_ATTRIBUTES', 'EARTH_SUN_DISTANCE')
        SE = data.metadata.get('IMAGE_ATTRIBUTES', 'SUN_ELEVATION')

        radiance = l7_radiance(img_data, QCMIN, QCMAX, RMIN, RMAX, src_nodata=0)
        reflectance = l7_reflectance(radiance, ESD, SE, int(band_idx), src_nodata=0.0)

        band_stack.append(reflectance)
        img.close()

    ndvi_data = ndvi(band_stack[0], band_stack[2])
    bi_data = bi(band_stack[3], band_stack[0], band_stack[2], band_stack[1])

    rgbn_data = np.array([ndvi_data, bi_data], dtype=np.float32)
       
    name = data.alias + '.tif'

    rgbn_path = write(rgbn_data, str(DIRS.proc / name), 
                      driver='GTiff', transform=transform, 
                      crs=crs)

    reproject = reproject_like(str(DIRS.proc / 'reference_image.tif'), rgbn_path, rgbn_path) # change with reproject like

    clip, transform = clip_raster(reproject, AIO)

    clip_path = write(clip, str(DIRS.proc / name), 
                      driver='GTiff', transform=transform, 
                      crs=WGS84)
    
    images.append(clip_path)

In [42]:
samples = gpd.read_file(str(DIRS.prep / 'mines.shp'))

features = samples.columns[:2]

lda = LinearDiscriminantAnalysis(solver='lsqr')

lda.fit(samples[features], samples.label)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='lsqr', store_covariance=False, tol=0.0001)

In [49]:
img = rasterio.open(images[0])
to_classify = img.read()

transform = img.transform

treecover = rasterio.open(treecover_2000, 'r')
cover = treecover.read(1)

to_classify[0][cover == 0] = 0.0
to_classify[1][cover == 0] = 0.0

In [50]:
px_matrix = to_classify.T.reshape((to_classify.shape[1]*to_classify.shape[2],to_classify.shape[0]))
df = pd.DataFrame.from_records(px_matrix, columns=features)

In [51]:
classified = lda.predict(df[features])

classified = np.reshape(classified, (to_classify.shape[2], to_classify.shape[1])).T
classified = classified.astype(np.uint8)

classified[cover == 0] = 0

test = write(classified, str(DIRS.proc / 'test.tif'), crs=WGS84, driver='GTiff',
             transform=transform, compress='lzw')

## 3. Results
#### 3.1 Tree cover

#### 3.2 Amber mining sites

In [45]:
data = gpd.read_file(str(DIRS.prep / 'mines.shp'))
data['color'] = data.apply(lambda row: 'green' if row['label'] == 0 else 'red',
                           axis=1)
data['name'] = data.apply(lambda row: 'Forest' if row['label'] == 0 else 'Mine',
                           axis=1)
data.drop('geometry', axis=1, inplace=True)

w = lda.coef_[0]
a = -w[0] / w[1]
xx = np.array([0.1,0.8])
yy = a * xx - (lda.intercept_[0]) / w[1]

src = ColumnDataSource(data)

scatter = figure(title='Spectral pattern of mines versus forest')

scatter.circle(x='ndvi', y='bi', color='color', legend='name', source=src)
scatter.line(x=xx, y=yy, color='black')

scatter.xaxis.axis_label = "NDVI"
scatter.yaxis.axis_label = "Bare Index"

show(scatter)

## 4. Discussion